In [ ]:
#Imports
import sys
sys.path.append('../../')

from pyspark.ml import Pipeline
from sparknlp.annotator import *
from sparknlp.base import DocumentAssembler, Finisher


In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[2]") \
    .config("spark.jar", "lib/sparknlp.jar") \
    .config("spark.driver.memory", "5g")\
    .config("spark.dirver.maxResultSize", "2g")\
    .getOrCreate()

In [ ]:
#Load the input data to be annotated
data = spark. \
        read. \
        parquet("../../../src/test/resources/sentiment.parquet"). \
        limit(1000)
data.cache()
data.count()
data.show()

In [ ]:
### Define the dataframe
document_assembler = DocumentAssembler() \
            .setInputCol("text")
        
### Transform input to appropriate schema
#assembled = document_assembler.transform(data)

In [ ]:
### Sentence detector
sentence_detector = SentenceDetectorModel() \
    .setInputCols(["document"]) \
    .setOutputCol("sentence")
#sentence_data = sentence_detector.transform(checked)

In [ ]:
### Tokenizer
tokenizer = RegexTokenizer() \
            .setInputCols(["sentence"]) \
            .setOutputCol("token")
#tokenized = tokenizer.transform(assembled)

In [ ]:
normalizer = Normalizer() \
            .setInputCols(["token"]) \
            .setOutputCol("normal")

In [ ]:
### Spell Checker
spell_checker = NorvigSweetingApproach() \
            .setInputCols(["normal"]) \
            .setOutputCol("spell")

#checked = spell_checker.fit(tokenized).transform(tokenized)

In [ ]:
sentiment_detector = ViveknSentimentApproach() \
    .setInputCols(["spell", "sentence"]) \
    .setOutputCol("sentiment") \
    .setPositiveSource("../../../src/test/resources/vivekn/positive") \
    .setNegativeSource("../../../src/test/resources/vivekn/negative") \
    .setPruneCorpus(False)


In [ ]:
finisher = Finisher() \
    .setInputCols(["sentiment"]) \
    .setIncludeKeys(True)

In [ ]:
pipeline = Pipeline(stages=[
    document_assembler,
    sentence_detector,
    tokenizer,
    normalizer,
    spell_checker,
    sentiment_detector,
    finisher
])

sentiment_data = pipeline.fit(data).transform(data)
sentiment_data.show()

In [ ]:
for r in sentiment_data.take(5):
    print(r)

In [ ]:
pipeline.write().overwrite().save("./ps")
pipeline.fit(data).write().overwrite().save("./ms")

In [ ]:
from pyspark.ml import Pipeline,PipelineModel

In [ ]:
Pipeline.read().load("./ps")
PipelineModel.read().load("./ms")